In [17]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import numpy as np
import evaluate

In [18]:
# --- 1. CARREGAR E CONSOLIDAR TODOS OS DADOS ANOTADOS MANUALMENTE ---

# Caminhos para todos os arquivos que você anotou
path_seed_set = '/Users/giossaurus/Developer/leia_tcc/data/processed/seed_set_anotado.csv'
path_cycle1 = '/Users/giossaurus/Developer/leia_tcc/data/processed/active_learning_cycle1.csv'
path_cycle2 = '/Users/giossaurus/Developer/leia_tcc/data/processed/active_learning_cycle2.csv'
# AJUSTE AQUI se o nome do seu último arquivo for diferente
path_lote_final = '/Users/giossaurus/Developer/leia_tcc/data/processed/anotacao_maior_final.csv' 

# Carregar todos os dataframes
df_seed = pd.read_csv(path_seed_set)
df_c1 = pd.read_csv(path_cycle1)
df_c2 = pd.read_csv(path_cycle2)
df_final = pd.read_csv(path_lote_final)

# Juntar todos em um único dataframe
df_full_manual = pd.concat([df_seed, df_c1, df_c2, df_final], ignore_index=True)

# Remover duplicatas caso algum exemplo tenha sido incluído em mais de um lote
df_full_manual.drop_duplicates(subset=['question_id'], inplace=True)

print(f"Total de exemplos anotados manualmente: {len(df_full_manual)}")

Total de exemplos anotados manualmente: 1001


In [19]:
# --- 2. PREPARAR O DATASET FINAL ---

# Vamos processar cada arquivo separadamente para evitar problemas de índice
all_data = []

print("Processando arquivos individuais...")

# Função auxiliar para processar cada dataframe
def process_dataframe(df, name):
    print(f"\n{name}:")
    print(f"  - Linhas totais: {len(df)}")
    print(f"  - Colunas disponíveis: {df.columns.tolist()}")
    
    # Mapear possíveis nomes de colunas
    text_cols = ['question', 'text']
    label_cols = ['intent_choice', 'label']
    
    # Encontrar a coluna de texto
    text_col = None
    for col in text_cols:
        if col in df.columns:
            text_col = col
            break
    
    # Encontrar a coluna de label
    label_col = None
    for col in label_cols:
        if col in df.columns:
            label_col = col
            break
    
    print(f"  - Coluna texto encontrada: {text_col}")
    print(f"  - Coluna label encontrada: {label_col}")
    
    if not text_col or not label_col:
        print(f"  - ERRO: Colunas essenciais não encontradas")
        return 0
    
    # Extrair dados diretamente sem operações de pandas que podem causar reindexing
    added_count = 0
    
    # Usar .iloc para evitar problemas de índice
    for i in range(len(df)):
        try:
            # Acessar por posição para evitar problemas de índice
            text_val = df.iloc[i][text_col]
            label_val = df.iloc[i][label_col]
            question_id = df.iloc[i].get('question_id', f"{name}_{i}")
            
            # Verificação mais permissiva
            if pd.notna(text_val) and pd.notna(label_val) and str(text_val).strip() and str(label_val).strip():
                all_data.append({
                    'text': str(text_val).strip(),
                    'label': str(label_val).strip(),
                    'question_id': str(question_id) if pd.notna(question_id) else f"{name}_{i}",
                    'source': name
                })
                added_count += 1
                
        except Exception as e:
            print(f"  - Erro na linha {i}: {e}")
            continue
    
    print(f"  - Exemplos adicionados: {added_count}")
    return added_count

# Processar cada arquivo
try:
    count1 = process_dataframe(df_seed, "Seed set")
except Exception as e:
    print(f"Erro no seed set: {e}")
    count1 = 0

try:
    count2 = process_dataframe(df_c1, "Cycle1")
except Exception as e:
    print(f"Erro no cycle1: {e}")
    count2 = 0

try:
    count3 = process_dataframe(df_c2, "Cycle2")
except Exception as e:
    print(f"Erro no cycle2: {e}")
    count3 = 0

try:
    count4 = process_dataframe(df_final, "Lote final")
except Exception as e:
    print(f"Erro no lote final: {e}")
    count4 = 0

print(f"\n=== RESUMO ===")
print(f"Total de exemplos extraídos: {len(all_data)}")
print(f"Por arquivo: Seed={count1}, Cycle1={count2}, Cycle2={count3}, Final={count4}")
print(f"Esperado: 1001, Atual: {len(all_data)}, Diferença: {1001 - len(all_data)}")

# Criar um novo dataframe limpo a partir dos dados extraídos
df_full_manual = pd.DataFrame(all_data)

# Mostrar distribuição por fonte
if 'source' in df_full_manual.columns:
    print(f"\nDistribuição por fonte:")
    print(df_full_manual['source'].value_counts())

# Remover duplicatas por question_id se disponível
if 'question_id' in df_full_manual.columns:
    initial_size = len(df_full_manual)
    df_full_manual = df_full_manual.drop_duplicates(subset=['question_id'])
    print(f"\nRemovidas {initial_size - len(df_full_manual)} duplicatas por question_id")

print(f"Total de exemplos após remoção de duplicatas: {len(df_full_manual)}")

# Manter apenas text e label para o modelo
df_final_clean = df_full_manual[['text', 'label']].copy()

# Verificar labels únicos antes do mapeamento
print(f"\nLabels únicos encontrados: {sorted(df_final_clean['label'].unique())}")

# Mapear rótulos para IDs
labels = df_final_clean['label'].unique().tolist()
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

# Aplicar mapeamento
df_final_clean['label'] = df_final_clean['label'].map(label2id)
unmapped_labels = df_final_clean['label'].isna().sum()
if unmapped_labels > 0:
    print(f"AVISO: {unmapped_labels} labels não puderam ser mapeados")

df_final_clean.dropna(subset=['label'], inplace=True)
df_final_clean['label'] = df_final_clean['label'].astype(int)

print(f"Labels mapeados: {labels}")
print(f"Dataset final limpo: {len(df_final_clean)} exemplos")

# Converter para Dataset do Hugging Face
from datasets import ClassLabel
full_dataset = Dataset.from_pandas(df_final_clean)

# Configurar ClassLabel para estratificação
features = full_dataset.features.copy()
features['label'] = ClassLabel(names=labels)
full_dataset = full_dataset.cast(features)

Processando arquivos individuais...

Seed set:
  - Linhas totais: 200
  - Colunas disponíveis: ['annotation_id', 'annotator', 'created_at', 'disciplina', 'id', 'intent_choice', 'lead_time', 'question', 'question_id', 'split', 'true_answer', 'updated_at']
  - Coluna texto encontrada: question
  - Coluna label encontrada: intent_choice
  - Exemplos adicionados: 200

Cycle1:
  - Linhas totais: 50
  - Colunas disponíveis: ['annotation_id', 'annotator', 'created_at', 'disciplina', 'id', 'intent_choice', 'lead_time', 'question', 'question_id', 'split', 'text', 'true_answer', 'uncertainty', 'updated_at']
  - Coluna texto encontrada: question
  - Coluna label encontrada: intent_choice
  - Exemplos adicionados: 50

Cycle2:
  - Linhas totais: 50
  - Colunas disponíveis: ['annotation_id', 'annotator', 'created_at', 'disciplina', 'id', 'intent_choice', 'lead_time', 'question', 'question_id', 'split', 'text', 'true_answer', 'uncertainty', 'updated_at']
  - Coluna texto encontrada: question
  - Colu

Casting the dataset:   0%|          | 0/999 [00:00<?, ? examples/s]

In [20]:

# --- 3. DIVISÃO ESTRATÉGICA: TREINO E TESTE ---
# Com 1000 exemplos, podemos criar um conjunto de teste robusto (20%)
dataset_split = full_dataset.train_test_split(test_size=0.2, stratify_by_column="label", seed=42)
train_dataset_full = dataset_split["train"]
test_dataset = dataset_split["test"]

# Dividir o treino novamente para ter um conjunto de validação durante o treino
train_val_split = train_dataset_full.train_test_split(test_size=0.2, stratify_by_column="label", seed=42)
train_dataset = train_val_split["train"]
eval_dataset = train_val_split["test"]

print(f"\nDataset de Treino: {len(train_dataset)} exemplos")
print(f"Dataset de Validação: {len(eval_dataset)} exemplos")
print(f"Dataset de Teste Final: {len(test_dataset)} exemplos")



Dataset de Treino: 639 exemplos
Dataset de Validação: 160 exemplos
Dataset de Teste Final: 200 exemplos


In [21]:
# --- 4. PREPARAR MODELO, TOKENIZER E TREINAMENTO ---

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": acc["accuracy"], "f1": f1["f1"]}

training_args = TrainingArguments(
    output_dir="/Users/giossaurus/Developer/leia_tcc/models/leia_classifier_1k_base",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5, # Com mais dados, 5 épocas é um bom ponto de partida
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

Map:   0%|          | 0/639 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/k4/x1x0j4qs0svb48fpvhhvkmbh0000gn/T/ipykernel_51419/4094535796.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
# --- 5. TREINAR E AVALIAR ---

print("\n--- INICIANDO TREINAMENTO DO MODELO DE BASE (1k) ---")
trainer.train()
print("\n--- TREINAMENTO CONCLUÍDO ---")

print("\n--- AVALIANDO PERFORMANCE FINAL NO CONJUNTO DE TESTE ---")
final_results = trainer.evaluate(eval_dataset=tokenized_test_dataset)

print("\n--- RESULTADOS FINAIS (NO CONJUNTO DE TESTE) ---")
print(f"Acurácia Final: {final_results['eval_accuracy']:.4f}")
print(f"F1-Score Final (Ponderado): {final_results['eval_f1']:.4f}")
print("-------------------------------------------------")

# Salvar o modelo final
trainer.save_model("/Users/giossaurus/Developer/leia_tcc/models/leia_classifier_1k_final")
print("Modelo final salvo em '/Users/giossaurus/Developer/leia_tcc/models/leia_classifier_1k_final'")


--- INICIANDO TREINAMENTO DO MODELO DE BASE (1k) ---


/opt/homebrew/Caskroom/miniforge/base/envs/torch-mps/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.988830,0.606250,0.488910
2,No log,0.826912,0.675000,0.613797
3,No log,0.767434,0.706250,0.678288
4,No log,0.750718,0.712500,0.686288
5,No log,0.759789,0.712500,0.685151


/opt/homebrew/Caskroom/miniforge/base/envs/torch-mps/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/homebrew/Caskroom/miniforge/base/envs/torch-mps/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/homebrew/Caskroom/miniforge/base/envs/torch-mps/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/homebrew/Caskroom/miniforge/base/envs/torch-mps/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then de


--- TREINAMENTO CONCLUÍDO ---

--- AVALIANDO PERFORMANCE FINAL NO CONJUNTO DE TESTE ---


/opt/homebrew/Caskroom/miniforge/base/envs/torch-mps/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)



--- RESULTADOS FINAIS (NO CONJUNTO DE TESTE) ---
Acurácia Final: 0.7200
F1-Score Final (Ponderado): 0.6982
-------------------------------------------------
Modelo final salvo em '/Users/giossaurus/Developer/leia_tcc/models/leia_classifier_1k_final'
